In [10]:
import pandas as pd
import matplotlib.pyplot as plt

veri = pd.read_excel("C:\\Users\\caglars\\Desktop\\da.xlsx")

In [24]:
import pandas as pd
import datetime as dt

veri['IslemTarih'] = pd.to_datetime(veri['IslemTarih'])
bugun = max(veri['IslemTarih'])
veri['Recency'] = (bugun - veri['IslemTarih']).dt.days

frequency = veri.groupby('UyeIsyeriAdi')['IslemID'].count().reset_index()
frequency.columns = ['UyeIsyeriAdi', 'Frequency']

monetary = veri.groupby('UyeIsyeriAdi')['IslemTutar'].sum().reset_index()
monetary.columns = ['UyeIsyeriAdi', 'Monetary']

rfm = pd.merge(frequency, monetary, on='UyeIsyeriAdi')
rfm = pd.merge(rfm, veri[['UyeIsyeriAdi', 'Recency']], on='UyeIsyeriAdi')
rfm = rfm.drop_duplicates(subset='UyeIsyeriAdi')

rfm['R_Score'] = pd.qcut(rfm['Recency'], q=5, labels=[5, 4, 3, 2, 1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'], q=5, labels=[1, 2, 3, 4, 5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], q=5, labels=[1, 2, 3, 4, 5])

rfm['RFM_Puan'] = rfm['R_Score'].astype(int) + rfm['F_Score'].astype(int) + rfm['M_Score'].astype(int)



def sınıflandırma(row):
    r_score = row['R_Score']
    f_score = row['F_Score']
    m_score = row['M_Score']
    
    if r_score >= 4 and f_score >= 4 and m_score >= 4:
        return 'Çok Değerli ve Sık İşlem Yapan (Segment 1)'
    elif r_score >= 3 and f_score >= 3 and m_score >= 3:
        return 'Değerli ve Orta Sık İşlem Yapan (Segment 2)'
    elif r_score >= 2 and f_score >= 2 and m_score >= 2:
        return 'Orta Değerli ve Az Sık İşlem Yapan (Segment 3)'
    else:
        return 'Diğer (Segment 4)'

rfm['Segment'] = rfm.apply(sınıflandırma, axis=1)

veri['IslemSaat'] = pd.to_datetime(veri['IslemTarih']).dt.hour
veri['IslemGun'] = pd.to_datetime(veri['IslemTarih']).dt.day_name()

uye_isyeri_kosullari = []

for isyeri_adi in veri['UyeIsyeriAdi'].unique():
    isyeri_verisi = veri[veri['UyeIsyeriAdi'] == isyeri_adi]
    en_az_islem_saatleri = isyeri_verisi['IslemSaat'].value_counts().idxmin()
    en_az_islem_gunleri = isyeri_verisi['IslemGun'].value_counts().idxmin()
    uye_isyeri_kosullari.append({'UyeIsyeriAdi': isyeri_adi, 'EnAzIslemSaatleri': en_az_islem_saatleri, 'EnAzIslemGunleri': en_az_islem_gunleri})

uye_isyeri_kosullari = pd.DataFrame(uye_isyeri_kosullari)

uyeadi = input("Lütfen bir UyeIsyeriAdi girin: ")

kampanya_sonucu = uye_isyeri_kosullari[uye_isyeri_kosullari['UyeIsyeriAdi'] == uyeadi]

if not kampanya_sonucu.empty:
    en_az_islem_saatleri = kampanya_sonucu.iloc[0]['EnAzIslemSaatleri']
    en_az_islem_gunleri = kampanya_sonucu.iloc[0]['EnAzIslemGunleri']
    print(f"{uyeadi} işyeri için kampanya koşulları: {en_az_islem_gunleri} günleri {en_az_islem_saatleri} - {en_az_islem_saatleri + 2} saatleri arasında komisyon indirimi mevcut.")
else:
    print(f"{uyeadi} işyeri için kampanya koşulları bulunamadı.")

Örek öz börek işyeri için kampanya koşulları: Sunday günleri 1 - 3 saatleri arasında komisyon indirimi mevcut.
